In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CLONG').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier, NaiveBayes, LogisticRegression
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer, StandardScaler, Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.sql.functions import year, month, dayofmonth, col, udf, length
from pyspark.sql.types import IntegerType
from pyspark.ml.clustering import KMeans
from pyspark.ml.recommendation import ALS

In [0]:
df = sqlContext.sql("SELECT * FROM smsspamcollection_1")
df = df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
df.describe().show()

+-------+-----+--------------------+
summary|class| text|
+-------+-----+--------------------+
 count| 5574| 5574|
 mean| null| 645.0|
 stddev| null| NaN|
 min| ham| &lt;#&gt; in mc...|
 max| spam|… we r stayin her...|
+-------+-----+--------------------+

In [0]:
dfb = df.withColumn('length',length(df['text']))
dfb.head(1)

Out[6]: [Row(class='ham', text='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', length=111)]

In [0]:
dfb.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [0]:
tkzr = Tokenizer(inputCol='text',outputCol='token_text')
stopr = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
countVec = CountVectorizer(inputCol='stop_tokens',outputCol='cvec')
idf =IDF(inputCol='cvec',outputCol='tfIdf')
classtonumeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
cleaner = VectorAssembler(inputCols=['tfIdf','length'], outputCol='features')

In [0]:
dfpipe = Pipeline(stages=[classtonumeric,tkzr,
                         stopr,countVec,idf,cleaner])

In [0]:
model = dfpipe.fit(dfb)

In [0]:
dfc = model.transform(dfb)

In [0]:
fdf = dfc.select('label','features')
fdf.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [0]:
train, test = fdf.randomSplit([.7,.3])

In [0]:
nb = NaiveBayes()
rfc = RandomForestClassifier()
gbt = GBTClassifier()
dtc = DecisionTreeClassifier()
lgr = LogisticRegression()

In [0]:
nbmod = nb.fit(train)
rfcmod = rfc.fit(train)
gbtmod = gbt.fit(train)
dtcmod = dtc.fit(train)
lgrmod = lgr.fit(train)

In [0]:
nbres = nbmod.transform(test)
rfcres = rfcmod.transform(test)
gbtres = gbtmod.transform(test)
dtcres = dtcmod.transform(test)
lgrres = lgrmod.transform(test)

In [0]:
accEval = MulticlassClassificationEvaluator()

In [0]:
print(accEval.evaluate(nbres))
print(accEval.evaluate(rfcres))
print(accEval.evaluate(gbtres))
print(accEval.evaluate(dtcres))
print(accEval.evaluate(lgrres))


0.9148471779618417
0.8069409568663131
0.9557645865832227
0.9462832471460435
0.9646531474812045